In [1]:
# start by importing the modules we will need
import requests
import json  #we will use these later in retrieving geodata and plotting university locations using openstreetmaps
import folium
import numpy as np
import cufflinks as cf
import pandas as pd #These allow us to open our data in a pandas data frame
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
print("Ready to go to college?".title())

Ready To Go To College?


In [2]:
CENTER_US = (39.8333333,-98.585522)
#london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=4)

In [3]:
import sys
from sys import exit

help(exit)

Help on built-in function exit in module sys:

exit(...)
    exit([status])
    
    Exit the interpreter by raising SystemExit(status).
    If the status is omitted or None, it defaults to zero (i.e., success).
    If the status is an integer, it will be used as the system exit status.
    If it is another kind of object, it will be printed and the system
    exit status will be one (i.e., failure).



In [4]:
#accepted_states = ['Alabama','Alaska','Arizona','Arkansas' ,'California' ,'Colorado','Connecticut', 'Delaware','Florida','Georgia']
state = input('Enter State: ex. NewYork ')
try:
    state = pd.read_csv(state)
except FileNotFoundError:
    print('Please re-run and follow the correct format. Enter the state in title format. If the state has two words, ignore the space.')
    print("For example: Connecticut, NewYork, NewJersey, etc.")
    try:
        state = input('Enter State: ex. NewYork ')
        state = pd.read_csv(state)
    except FileNotFoundError:
        exit(0)
    

    #while True:
        #state = input('Enter State: ex. NewYork ')
        #if state is not in accepted_states:
            #break
        #else:
            #state = pd.read_csv(state)
#Add column based on size
state['size'] = np.nan
state['size'][state['Student population']>15000] = 'large'
state['size'][state['Student population']<=15000] = 'medium'
state['size'][state['Student population'] <=5000] = 'small'
state

display(state)
#filter by type of institution
college_types = list(state['Type'].value_counts().index)
i = 1
for types in college_types:
    print(i,types)
    i = i + 1
choice = input ('What type of university are you interested in? Enter the correspondong number (1 - 6). If you have no preference, just hit enter. ')
try:
    if choice == '':
        state2 = state
    else:
        choice = int(choice)#convert choice into an integer that corresponds to the menu options
        choice_value = college_types[choice-1]
        state2 = state[state['Type']==choice_value]
    display(state2)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state2 = state

#filter by surrounding area
surrounding_area = list(state2['Campus setting'].value_counts().index)
i = 1
for types in surrounding_area:
    print(i,types)
    i = i + 1
area = input ('What setting would you like the campus to be in? Please choose from the menu by entering the corresponding number or press enter if you have no preference.')
try:
    if area == '':
        state3 = state2
    else:
        area = int(area)
        setting_preference = surrounding_area[area-1]
        state3 = state2[state2['Campus setting']==setting_preference]
    display(state3)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state3=state2

#filter by size
college_sizes = list(state3['size'].value_counts().index)
i = 1
for types in college_sizes:
    print(i,types)
    i = i + 1
size_preference = input("How big do you want your university to be? Above are your options. Press enter if you have no preference.")
try:
    if size_preference == '':
        state4 = state3
    else:
        size_preference= int(size_preference) 
        size = college_sizes[size_preference -1]
        state4=state3[state3['size']==size]
    display(state4)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state4=state3

#filter by on campus housing
campus_housing = input('Do you want campus housing? Yes or no or type anything you do not have a preference. ')
campus_housing = campus_housing.capitalize()

if campus_housing == 'Yes':
    state5 = state4[state4['Campus housing'] == 'Yes']
    display(state5)
elif campus_housing == 'No':
    state5 = state4[state4['Campus housing'] == 'No']
    display(state5)
else: 
    state5 = state4
    display(state5)    


Enter State: ex. NewYork Ohio


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
0,Allegheny Wesleyan College,"2161 Woodsdale Rd, Salem, Ohio 44460-9598",awc.edu,"4-year, Private not-for-profit",Less than one year certificate|Associate's deg...,Rural: Fringe,Yes,79.0,79.0,18%,27%,Fall 2010,"$5,247",-,200873.0,3457300.0,small
1,Antioch College,"1 Morgan Pl, Yellow Springs, Ohio 45387",antiochcollege.edu,"4-year, Private not-for-profit",Bachelor's degree,Town: Fringe,Yes,223.0,223.0,NaN,NaN,Fall 2010,"$15,182",-,483018.0,4244300.0,small
2,Antioch University Online,"900 Dayton Street, Yellow Springs, Ohio 45387",online.antioch.edu,"4-year, Private not-for-profit",Bachelor's degree,Rural: Fringe,No,163.0,163.0,NaN,NaN,Fall 2010,NaN,-,485908.0,NaN,small
3,Antioch University-Midwest,"900 Dayton St, Yellow Springs, Ohio 45387",midwest.antioch.edu/,"4-year, Private not-for-profit",Bachelor's degree|Postbaccalaureate certificat...,Rural: Fringe,No,156.0,38.0,NaN,NaN,Fall 2010,NaN,-,245892.0,301000.0,small
4,Art Academy of Cincinnati,"1212 Jackson Street, Cincinnati, Ohio 45202-1706",www.artacademy.edu,"4-year, Private not-for-profit",Associate's degree|Bachelor's degree|Master's ...,City: Large,Yes,207.0,201.0,52%,NaN,Fall 2010,"$24,137",-,201061.0,301100.0,small
5,Ashland University,"401 College Avenue, Ashland, Ohio 44805",https://www.ashland.edu,"4-year, Private not-for-profit",Less than one year certificate|One but less th...,Town: Fringe,Yes,6579.0,4814.0,63%,29%,Fall 2010,"$20,601",-,201104.0,301200.0,medium
6,Aultman College of Nursing and Health Sciences,"2600 6th St SW, Canton, Ohio 44710-1797",www.aultmancollege.edu,"4-year, primarily associate's, Private not-for...",Associate's degree|Bachelor's degree,City: Small,No,391.0,391.0,25%,25%,Fall 2010,"$17,300",-,201177.0,648700.0,small
7,Baldwin Wallace University,"275 Eastland Rd, Berea, Ohio 44017-2088",https://www.bw.edu,"4-year, Private not-for-profit",Less than one year certificate|Bachelor's degr...,Suburb: Large,Yes,3933.0,3287.0,67%,NaN,Fall 2010,"$22,326",-,201195.0,301400.0,small
8,Bluffton University,"1 University Drive, Bluffton, Ohio 45817-2104",www.bluffton.edu,"4-year, Private not-for-profit",Bachelor's degree|Postbaccalaureate certificat...,Town: Distant,Yes,952.0,865.0,49%,39%,Fall 2010,"$25,101",-,201371.0,301600.0,small
9,Bowling Green State University-Main Campus,"220 McFall Ctr, Bowling Green, Ohio 43403",www.bgsu.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,Town: Fringe,Yes,17644.0,14852.0,55%,31%,Fall 2010,"$17,313",-,201441.0,301800.0,large


1 4-year, Private not-for-profit
2 4-year, Public
3 4-year, primarily associate's, Public
4 4-year, primarily associate's, Private not-for-profit
5 4-year, primarily associate's, Private for-profit
6 4-year, Private for-profit
What type of university are you interested in? Enter the correspondong number (1 - 6). If you have no preference, just hit enter. 2


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
9,Bowling Green State University-Main Campus,"220 McFall Ctr, Bowling Green, Ohio 43403",www.bgsu.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,Town: Fringe,Yes,17644.0,14852.0,55%,31%,Fall 2010,"$17,313",-,201441.0,301800.0,large
20,Central State University,"1400 Brush Row Rd, Wilberforce, Ohio 45384-1004",www.centralstate.edu,"4-year, Public",Bachelor's degree,Rural: Fringe,Yes,1741.0,1729.0,26%,2%,Fall 2010,"$10,942",-,201690.0,302600.0,small
26,Cleveland State University,"2121 Euclid Avenue, Cleveland, Ohio 44115-2214",www.csuohio.edu/,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Large,Yes,16864.0,12352.0,41%,NaN,Fall 2010,"$15,555",-,202134.0,303200.0,large
45,Kent State University at Kent,"Executive Office, 2nd Floor Library, Kent, Ohi...",www.kent.edu,"4-year, Public",Less than one year certificate|One but less th...,Suburb: Large,Yes,30167.0,23684.0,55%,7%,Fall 2010,"$18,180",-,203517.0,305100.0,large
57,Miami University-Hamilton,"1601 University Blvd, Hamilton, Ohio 45011",miamioh.edu/regionals/,"4-year, Public",One but less than two years certificate|Associ...,Suburb: Large,No,2894.0,2884.0,23%,NaN,Fall 2010,"$9,649",-,204006.0,307702.0,small
58,Miami University-Middletown,"4200 E University Blvd, Middletown, Ohio 45042",miamioh.edu/regionals/,"4-year, Public",One but less than two years certificate|Associ...,Suburb: Small,No,1914.0,1914.0,15%,NaN,Fall 2010,"$10,219",-,204015.0,307701.0,small
59,Miami University-Oxford,"501 East High St, Oxford, Ohio 45056",miamioh.edu,"4-year, Public",One but less than two years certificate|Associ...,Town: Fringe,Yes,19697.0,16981.0,78%,NaN,Fall 2010,"$20,186",-,204024.0,307700.0,large
70,Ohio State University-Main Campus,"190 N. Oval Mall, Columbus, Ohio 43210",www.osu.edu/,"4-year, Public",One but less than two years certificate|Associ...,City: Large,Yes,59482.0,45831.0,84%,9%,Fall 2010,"$17,971",-,204796.0,309000.0,large
74,Ohio University-Main Campus,"020 Chubb Hall, Athens, Ohio 45701-2979",www.ohio.edu,"4-year, Public",Less than one year certificate|Associate's deg...,Town: Distant,Yes,29509.0,23585.0,64%,25%,Fall 2010,"$21,899",-,204857.0,310000.0,large
79,Shawnee State University,"940 Second St, Portsmouth, Ohio 45662",www.shawnee.edu/,"4-year, Public",One but less than two years certificate|Associ...,Town: Fringe,Yes,3772.0,3621.0,30%,31%,Fall 2010,"$13,632",-,205443.0,994200.0,small


1 City: Large
2 Suburb: Large
3 Town: Fringe
4 Rural: Fringe
5 Town: Distant
6 Suburb: Small
7 City: Midsize
8 City: Small
What setting would you like the campus to be in? Please choose from the menu by entering the corresponding number or press enter if you have no preference.


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
9,Bowling Green State University-Main Campus,"220 McFall Ctr, Bowling Green, Ohio 43403",www.bgsu.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,Town: Fringe,Yes,17644.0,14852.0,55%,31%,Fall 2010,"$17,313",-,201441.0,301800.0,large
20,Central State University,"1400 Brush Row Rd, Wilberforce, Ohio 45384-1004",www.centralstate.edu,"4-year, Public",Bachelor's degree,Rural: Fringe,Yes,1741.0,1729.0,26%,2%,Fall 2010,"$10,942",-,201690.0,302600.0,small
26,Cleveland State University,"2121 Euclid Avenue, Cleveland, Ohio 44115-2214",www.csuohio.edu/,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Large,Yes,16864.0,12352.0,41%,NaN,Fall 2010,"$15,555",-,202134.0,303200.0,large
45,Kent State University at Kent,"Executive Office, 2nd Floor Library, Kent, Ohi...",www.kent.edu,"4-year, Public",Less than one year certificate|One but less th...,Suburb: Large,Yes,30167.0,23684.0,55%,7%,Fall 2010,"$18,180",-,203517.0,305100.0,large
57,Miami University-Hamilton,"1601 University Blvd, Hamilton, Ohio 45011",miamioh.edu/regionals/,"4-year, Public",One but less than two years certificate|Associ...,Suburb: Large,No,2894.0,2884.0,23%,NaN,Fall 2010,"$9,649",-,204006.0,307702.0,small
58,Miami University-Middletown,"4200 E University Blvd, Middletown, Ohio 45042",miamioh.edu/regionals/,"4-year, Public",One but less than two years certificate|Associ...,Suburb: Small,No,1914.0,1914.0,15%,NaN,Fall 2010,"$10,219",-,204015.0,307701.0,small
59,Miami University-Oxford,"501 East High St, Oxford, Ohio 45056",miamioh.edu,"4-year, Public",One but less than two years certificate|Associ...,Town: Fringe,Yes,19697.0,16981.0,78%,NaN,Fall 2010,"$20,186",-,204024.0,307700.0,large
70,Ohio State University-Main Campus,"190 N. Oval Mall, Columbus, Ohio 43210",www.osu.edu/,"4-year, Public",One but less than two years certificate|Associ...,City: Large,Yes,59482.0,45831.0,84%,9%,Fall 2010,"$17,971",-,204796.0,309000.0,large
74,Ohio University-Main Campus,"020 Chubb Hall, Athens, Ohio 45701-2979",www.ohio.edu,"4-year, Public",Less than one year certificate|Associate's deg...,Town: Distant,Yes,29509.0,23585.0,64%,25%,Fall 2010,"$21,899",-,204857.0,310000.0,large
79,Shawnee State University,"940 Second St, Portsmouth, Ohio 45662",www.shawnee.edu/,"4-year, Public",One but less than two years certificate|Associ...,Town: Fringe,Yes,3772.0,3621.0,30%,31%,Fall 2010,"$13,632",-,205443.0,994200.0,small


1 large
2 small
3 medium
How big do you want your university to be? Above are your options. Press enter if you have no preference.1


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
9,Bowling Green State University-Main Campus,"220 McFall Ctr, Bowling Green, Ohio 43403",www.bgsu.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,Town: Fringe,Yes,17644.0,14852.0,55%,31%,Fall 2010,"$17,313",-,201441.0,301800.0,large
26,Cleveland State University,"2121 Euclid Avenue, Cleveland, Ohio 44115-2214",www.csuohio.edu/,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Large,Yes,16864.0,12352.0,41%,NaN,Fall 2010,"$15,555",-,202134.0,303200.0,large
45,Kent State University at Kent,"Executive Office, 2nd Floor Library, Kent, Ohi...",www.kent.edu,"4-year, Public",Less than one year certificate|One but less th...,Suburb: Large,Yes,30167.0,23684.0,55%,7%,Fall 2010,"$18,180",-,203517.0,305100.0,large
59,Miami University-Oxford,"501 East High St, Oxford, Ohio 45056",miamioh.edu,"4-year, Public",One but less than two years certificate|Associ...,Town: Fringe,Yes,19697.0,16981.0,78%,NaN,Fall 2010,"$20,186",-,204024.0,307700.0,large
70,Ohio State University-Main Campus,"190 N. Oval Mall, Columbus, Ohio 43210",www.osu.edu/,"4-year, Public",One but less than two years certificate|Associ...,City: Large,Yes,59482.0,45831.0,84%,9%,Fall 2010,"$17,971",-,204796.0,309000.0,large
74,Ohio University-Main Campus,"020 Chubb Hall, Athens, Ohio 45701-2979",www.ohio.edu,"4-year, Public",Less than one year certificate|Associate's deg...,Town: Distant,Yes,29509.0,23585.0,64%,25%,Fall 2010,"$21,899",-,204857.0,310000.0,large
89,University of Akron Main Campus,"302 Buchtel Common, Akron, Ohio 44325-4702",www.uakron.edu,"4-year, Public",Less than one year certificate|One but less th...,City: Midsize,Yes,21100.0,17417.0,38%,35%,Fall 2010,"$16,568",-,200800.0,312300.0,large
92,University of Cincinnati-Main Campus,"2624 Clifton Avenue, Cincinnati, Ohio 45221-0063",www.uc.edu,"4-year, Public",Less than one year certificate|One but less th...,City: Large,Yes,36596.0,25820.0,67%,19%,Fall 2010,"$19,978",-,201885.0,312500.0,large
98,University of Toledo,"2801 W Bancroft, Toledo, Ohio 43606-3390",www.utoledo.edu/,"4-year, Public",Less than one year certificate|One but less th...,City: Large,Yes,20615.0,16223.0,43%,NaN,Fall 2010,"$15,919",-,206084.0,313100.0,large
106,Wright State University-Main Campus,"3640 Colonel Glenn Highway, Dayton, Ohio 45435...",www.wright.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,Suburb: Large,Yes,16655.0,12504.0,36%,NaN,Fall 2010,"$14,464",-,206604.0,307800.0,large


Do you want campus housing? Yes or no or type anything you do not have a preference. yes


,Name,Address,Website,Type,Awards offered,Campus setting,Campus housing,Student population,Undergraduate students,Graduation Rate,Transfer-Out Rate,Cohort Year *,Net Price **,Largest Program,IPEDS ID,OPE ID,size
9,Bowling Green State University-Main Campus,"220 McFall Ctr, Bowling Green, Ohio 43403",www.bgsu.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,Town: Fringe,Yes,17644.0,14852.0,55%,31%,Fall 2010,"$17,313",-,201441.0,301800.0,large
26,Cleveland State University,"2121 Euclid Avenue, Cleveland, Ohio 44115-2214",www.csuohio.edu/,"4-year, Public",Bachelor's degree|Postbaccalaureate certificat...,City: Large,Yes,16864.0,12352.0,41%,NaN,Fall 2010,"$15,555",-,202134.0,303200.0,large
45,Kent State University at Kent,"Executive Office, 2nd Floor Library, Kent, Ohi...",www.kent.edu,"4-year, Public",Less than one year certificate|One but less th...,Suburb: Large,Yes,30167.0,23684.0,55%,7%,Fall 2010,"$18,180",-,203517.0,305100.0,large
59,Miami University-Oxford,"501 East High St, Oxford, Ohio 45056",miamioh.edu,"4-year, Public",One but less than two years certificate|Associ...,Town: Fringe,Yes,19697.0,16981.0,78%,NaN,Fall 2010,"$20,186",-,204024.0,307700.0,large
70,Ohio State University-Main Campus,"190 N. Oval Mall, Columbus, Ohio 43210",www.osu.edu/,"4-year, Public",One but less than two years certificate|Associ...,City: Large,Yes,59482.0,45831.0,84%,9%,Fall 2010,"$17,971",-,204796.0,309000.0,large
74,Ohio University-Main Campus,"020 Chubb Hall, Athens, Ohio 45701-2979",www.ohio.edu,"4-year, Public",Less than one year certificate|Associate's deg...,Town: Distant,Yes,29509.0,23585.0,64%,25%,Fall 2010,"$21,899",-,204857.0,310000.0,large
89,University of Akron Main Campus,"302 Buchtel Common, Akron, Ohio 44325-4702",www.uakron.edu,"4-year, Public",Less than one year certificate|One but less th...,City: Midsize,Yes,21100.0,17417.0,38%,35%,Fall 2010,"$16,568",-,200800.0,312300.0,large
92,University of Cincinnati-Main Campus,"2624 Clifton Avenue, Cincinnati, Ohio 45221-0063",www.uc.edu,"4-year, Public",Less than one year certificate|One but less th...,City: Large,Yes,36596.0,25820.0,67%,19%,Fall 2010,"$19,978",-,201885.0,312500.0,large
98,University of Toledo,"2801 W Bancroft, Toledo, Ohio 43606-3390",www.utoledo.edu/,"4-year, Public",Less than one year certificate|One but less th...,City: Large,Yes,20615.0,16223.0,43%,NaN,Fall 2010,"$15,919",-,206084.0,313100.0,large
106,Wright State University-Main Campus,"3640 Colonel Glenn Highway, Dayton, Ohio 45435...",www.wright.edu,"4-year, Public",Less than one year certificate|Bachelor's degr...,Suburb: Large,Yes,16655.0,12504.0,36%,NaN,Fall 2010,"$14,464",-,206604.0,307800.0,large


In [7]:
school_data= pd.DataFrame({'Name':state4['Name'],
              'Location':state4['Address'],'Website': state4['Website']})
display(school_data)
#list(school_data)

#created a new data frame with the information we want. 



def get_coordinates_using_google(location):
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    options = { 'address' : location }  
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = geodata['results'][0]['geometry']['location']
    return coords
try:
    for index, each_school in school_data.iterrows():
        location = each_school['Location']
        coords = get_coordinates_using_google(location)
        name = str(each_school['Name'])
        website = str(each_school['Website'])
        #print(name, website, coords)
        pos = (coords['lat'],coords['lng'])
        message = "%s: %s" % (name,website)
        i=i+1
        marker = folium.CircleMarker(location=pos,
                                    popup=message,
                                    color='#3226cc'
                                    )
        map.add_children(marker)
    display(map)  
except IndexError:
    print("Try running me again.")


,Location,Name,Website
9,"220 McFall Ctr, Bowling Green, Ohio 43403",Bowling Green State University-Main Campus,www.bgsu.edu
26,"2121 Euclid Avenue, Cleveland, Ohio 44115-2214",Cleveland State University,www.csuohio.edu/
45,"Executive Office, 2nd Floor Library, Kent, Ohi...",Kent State University at Kent,www.kent.edu
59,"501 East High St, Oxford, Ohio 45056",Miami University-Oxford,miamioh.edu
70,"190 N. Oval Mall, Columbus, Ohio 43210",Ohio State University-Main Campus,www.osu.edu/
74,"020 Chubb Hall, Athens, Ohio 45701-2979",Ohio University-Main Campus,www.ohio.edu
89,"302 Buchtel Common, Akron, Ohio 44325-4702",University of Akron Main Campus,www.uakron.edu
92,"2624 Clifton Avenue, Cincinnati, Ohio 45221-0063",University of Cincinnati-Main Campus,www.uc.edu
98,"2801 W Bancroft, Toledo, Ohio 43606-3390",University of Toledo,www.utoledo.edu/
106,"3640 Colonel Glenn Highway, Dayton, Ohio 45435...",Wright State University-Main Campus,www.wright.edu


In [8]:
i = 0

all_data= pd.DataFrame({'Name':state['Name'],
              'Location':state['Address'],'Website': state['Website']})
def get_coordinates_using_google(location):
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    options = { 'address' : location }  
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = geodata['results'][0]['geometry']['location']
    return coords
try:
    for index, each_school in all_data.iterrows():
        location = each_school['Location']
        coords = get_coordinates_using_google(location)
        name = str(each_school['Name'])
        website = str(each_school['Website'])
        #print(name, website, coords)
        pos = (coords['lat'],coords['lng'])
        message = "%s: %s" % (name,website)
        i=i+1
        marker = folium.CircleMarker(location=pos,
                                    popup=message,
                                    color='#3226cc'
                                    )
        map.add_children(marker)
    display(map)  
except IndexError:
    print("Try running me again.")


Try running me again.
